# Advanced SQL for Data Scientists (PostgreSQL)

## Chapter 1: Overview

### Data Management Operations
* Linking data from different data stores
* Filtering and reformatting data for different uses
* Aggregating data to provide 'big picture' summaries
* Answering specific questions about business operations

### Data Sources
* Relational Databases
* NoSQL Database:
    * Non-relational databases: NoSQL
* Manually Managed Data

### Types of SQL Commands

```sql
CREATE TABLE address_book (
    pk_id PRIMARY KEY, 
    first_name VARCHAR(20),
    last_name VARCHAR (20),
    address VARCHAR (200)
    
CREATE TABLE order_book (
    order_id PRIMARY KEY, 
    f_pk_id VARCHAR(20),
    item VARCHAR (20),
    quantity INTEGER
```
#### Data Manipulation
  * Used to consolidate/clean/view information prior to using
  * Common Commands:
    * INSERT: Adds one row to a table
    * UPDATE: Updates row(s) with a new value(s)
    * DELETE: Deletes row(s) from a table
    * SELECT: Selects information from a database

#### Data Definition
  * Used to define structures for organizing data in a database
  * Data Structures: 
    * Tables: Collections of related data records
    * Indexes: Sets of data about the locations of records
    * Views: Repeated information derived from one or more tables
        * Helps showcase/retrieve data without unnecessary results
    * Schemas: Collections of all of the above
 * Common Commands:
   * CREATE TABLE: Defined above
   * CREATE INDEX: Builds an index to quickly look up rows and tables
```sql
CREATE INDEX idx_last_name
    ON address_book
    USING (last_name)
```
    * CREATE VIEW: Defined above
```sql
CREATE VIEW comb_sales AS
    SELECT (ob.quantity, ob.item, ab.first_name)
    FROM order_book ob
    LEFT JOIN address_book ab
    ON ab.pk_id = ob.f_pk_id
```
  * CREATE SCHEMA: Create a space to organize related structures

```sql
  CREATE SCHEMA data_sci
```

```
  ____________Schema___________
|                              |
|   ___________                |
|  | Table     |               |
|  |  A Index  |               |
|  |  B Index  |               |
|  |___________|  ----> View   |
|  | Table     |               |
|  |  C Index  |               |
|  |  D Index  |               |
|  |___________|               |
|                              |
|______________________________|

```

---

## Chapter 2: Basis Statistics

* Example Table Schema
```sql
CREATE TABLE staff (
    id PRIMARY KEY, 
    last_name VARCHAR(100),
    email VARCHAR (200),
    gender VARCHAR (10),
    department VARCHAR (100),
    start_date DATE,
    salary INTEGER,
    job_title VARCHAR (100),
    region_id INTEGER
)
```

### Aggregate Function Fun

* How many people does each department have?
```sql
SELECT
    department, 
    COUNT(department)
FROM
    staff
GROUP BY
    department
```

* What is the salary of the highest paid employee?
```sql
SELECT
    MAX(salary)
FROM staff
```

* What is the highest paying salary per department?
```sql
SELECT
    department,
    MAX(salary)
FROM
    staff
GROUP BY
    department
```

### Statistical Function Fun
* What is the average salary paid per employee in each department?
```sql
SELECT department, avg(salary) FROM staff GROUP BY department
```

* Do the above with variance and stdevs for the spread!
```sql
SELECT
    department,
    avg(salary),
    var_pop(salary),
    stddev_pop(salary)
FROM
    staff
GROUP BY
    department
```

### Filtering and Grouping Fun
* What are the top 10, highest-paying employees between 50,000 and 100,000 that are not in Grocery?

```sql
SELECT
	last_name, department, salary
FROM
	staff 
WHERE 
    (salary < 100000) AND
    (salary > 50000) AND
    (department != 'Grocery')
ORDER BY
	salary DESC LIMIT 10
```

---

### Chapter 3: Data Manipulation

* Example Table Schema
```sql
CREATE TABLE staff (
    id PRIMARY KEY, 
    last_name VARCHAR(100),
    email VARCHAR (200),
    gender VARCHAR (10),
    department VARCHAR (100),
    start_date DATE,
    salary INTEGER,
    job_title VARCHAR (100),
    region_id INTEGER
)
```

#### Reformatting Character Data

* Return all unique departments in upper case
```sql
SELECT
    DISTINCT UPPER(department)
FROM
    staff
```

* Return all unique department - job title combinations in lower case
```sql
SELECT
	DISTINCT(LOWER(department || ' - ' || job_title))
FROM
	staff
```

#### Filtering Data
* Create a table that lists the job title and if it has 'Assistant' in its name
```sql
SELECT
	DISTINCT job_title,
    (job_title LIKE '%Assistant%')
FROM
	staff
```

* Get the second unique word from all job titles whose job title includes 'Assistant'
```sql
SELECT
	TRIM(SUBSTRING(job_title, 10), '')
FROM
	staff
WHERE
	job_title LIKE 'Assistant%'
```

* Replace all job titles whose first name is 'Assistant' with 'Asst.'
```sql
SELECT
	OVERLAY(job_title PLACING 'Asst.' FROM 1 FOR 9)
FROM
	staff
WHERE
	job_title LIKE 'Assistant%'
```

* Find all job titles with Assistant and the levels III or IV via regex'
```sql
SELECT
	job_title
FROM
	staff
WHERE
	job_title SIMILAR TO 'Assistant%(III|IV)'
```

* Find all job titles starting with E, P, or S
```sql
SELECT
  job_title
FROM
  staff
WHERE
  job_title SIMILAR TO '[E,P,S]%'
```

#### Reformatting Numbers
* Get the average salary from each department rounded to 2 decimal places. 
```sql
SELECT
    department,
    ROUND(AVG(salary), 2)
FROM
    staff
GROUP BY
    department
```

* Get the average salary from each department with no decimal places and without rounding. 
```sql
SELECT
    department,
    TRUNC(AVG(salary), 2)
FROM
    staff
GROUP BY
    department
---

### Chapter 4: Data Filtering, Joining, and Aggregation

* Example Table Schema
```sql
CREATE TABLE staff (
    id PRIMARY KEY, 
    last_name VARCHAR(100),
    email VARCHAR (200),
    gender VARCHAR (10),
    department VARCHAR (100),
    start_date DATE,
    salary INTEGER,
    job_title VARCHAR (100),
    region_id INTEGER
)
```

```sql
CREATE TABLE company_divisions (
    department VARCHAR(100),
    company_division VARCHAR(100)
)

```

```sql
CREATE TABLE company_regions (
    region_id PRIMARY KEY,
    company_regions VARCHAR(20),
    country VARCHAR (20)
) 
```

#### Subqueries in SELECT clauses
* Return a table showcasing an employee's last name, department, salary, and avg salary for the dept
    * You need to make sure that you distinguish between the lower and the upper levels of a table via aliasing!
    * SQL is dumb and can't tell the difference!

```sql
SELECT 
	s1.last_name,
	s1.department,
	s1.salary,
	(SELECT(round(avg(salary), 2)) FROM staff s2 WHERE s2.department = s1.department)
FROM
	staff s1
```

#### Subqueries in FROM clauses

* Find the average executive salary by department
    * You can use subqueries where you would normally use tables in the from clause
```sql
SELECT
	s1.department,
	round(avg(s1.salary), 2)
FROM
(SELECT
	department,
	salary
FROM
	staff
WHERE
	salary > 100000) s1
GROUP BY
	s1.department
```

#### Subqueries in WHERE clauses

* Find the department of the person with the highest salary
```sql
SELECT
	s1.last_name,
	s1.department,
	s1.salary
FROM
	staff s1
WHERE
	s1.salary =
	(SELECT 
		MAX(s2.salary)
	FROM
		staff s2)
```

#### Creating a Join Statement
* Create a view named staff_orgs showcasing an employee's last name, department, and company division
    * Views save you time since you don't have to enter it in again and again!

```sql
CREATE VIEW staff_orgs AS
SELECT
	s.last_name,
	s.department,
	cd.company_division
FROM
	staff s
LEFT JOIN
	company_divisions cd
	ON cd.department = s.department
```

#### Creating Views
* From the view staff_orgs get a count of the number of employees per company_division
```sql
SELECT
	department,
	COUNT(company_division)
FROM
	staff_orgs
GROUP BY
	department
```

* Create a view staff_orgs_region with all of staff, company_division and company_regions
```sql
CREATE VIEW staff_orgs_region
SELECT
	s.*,
	cd.company_division,
	cr.company_regions,
	cr.country
FROM
	staff s
LEFT JOIN
	company_divisions cd
	ON cd.department = s.department
LEFT JOIN
	company_regions
	ON s.region_id, cr.region_id
```

#### Creating Subtotals
* Select the number of employees by company_region, and country from staff_orgs_region
    * Grouping Sets: Group of columns by which you group data
    * ROLL UP: Calculates sums in the hierarchy for whatever variables are placed in it

```sql
SELECT
	company_regions, 
	country,
	count(*)
FROM
	staff_orgs_region
GROUP BY
	ROLLUP(country, company_regions)
ORDER BY
	country, company_regions
```

* For each region, show the count breakdown by company_division
    * CUBE: Creates all possible combinatoins of sets of grouping columns

```sql
SELECT
	company_regions, 
	company_division,
	count(*) cnt
FROM
	staff_orgs_region
GROUP BY
	CUBE(company_division, company_regions)
ORDER BY
	company_regions DESC,
	cnt
```

#### Getting only the top searches
* List the employees with the top 10 salaries using only FETCH FIRST
    * FETCH FIRST sorts and limits result
        * Ordering is performed before selecting the rows to return
        * FETCH FIRST is supported by all while as LIMIT may not be!

```sql
SELECT
	last_name,
	salary
FROM
	staff
ORDER BY
	salary DESC
FETCH FIRST 
	10 ROWS ONLY
```

---

### Chapter 5: Window Functions

* Window Functions allow us to make SQL statements about rows that are related to the current row during processing. 

* Example Table Schema
```sql
CREATE TABLE staff (
    id PRIMARY KEY, 
    last_name VARCHAR(100),
    email VARCHAR (200),
    gender VARCHAR (10),
    department VARCHAR (100),
    start_date DATE,
    salary INTEGER,
    job_title VARCHAR (100),
    region_id INTEGER
)
```

```sql
CREATE TABLE company_divisions (
    department VARCHAR(100),
    company_division VARCHAR(100)
)

```

```sql
CREATE TABLE company_regions (
    region_id PRIMARY KEY,
    company_regions VARCHAR(20),
    country VARCHAR (20)
) 
```

#### OVER PARTITION
* Return the average salary for each employee's department alongside their own salary using OVER PARTITION
    * OVER PARTITION allows for you to do both grouping and sorting in one go vs. using a subquery!
    * OVER PARTITION divides the resulting set into partitions then performs computations on each subset of paritioned data

```sql
SELECT 
	department,
	last_name,
	salary,
	avg(salary) OVER (PARTITION BY department)
FROM
	staff
```
#### FIRST_VALUE

* Return the (alphabetically-ordered) first person's salary for each employee's department alongside their own salary using FIRST_VALUE.
    * FIRST_VALUE allows for you to group and just return the first value when used in conjunction with an OVER PARTITION
    * Consider it as another aggregate function that retrieves the first element of a list
    * Most useful when used in conjunction with an ORDER BY
    
```sql
SELECT
	department,
	last_name,
	salary,
	FIRST_VALUE(salary) OVER (PARTITION BY department ORDER BY last_name)
FROM
	staff
```

#### RANK

* Return the max salary for each employee's department alongside their own salary with salary ranking. 
    * RANK works with OVER PARTITION to order results and assigns a rank value based on the way that the partition data is sorted
    * RANK will restart as it goes on to the next partition column value
    * This streamlines subqueries that would otherwise take multiple queries to process

```sql
SELECT
    department,
    last_name,
    salary
    rank() OVER (PARTITION BY department ORDER BY salary)
```
#### LAG and LEAD
* Return the difference between each employee's salary with the person who least makes more than them alongside their own salary and department information . 
    * LAG and LEAD are functions that reference + or - the currently processed row
    * LAG's first row (or however many it's lagged by) will be a null value
        * Same logic for the LEAD with the bottom of the partition

```sql
SELECT
	s1.*,
	(s1.lagged_sal - s1.salary) sal_diffs
FROM
(SELECT
	department,
	last_name,
	salary,
	lag(salary) OVER (PARTITION BY department ORDER BY salary DESC) lagged_sal
 FROM
 	staff) s1
```

#### NTILE